In [431]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
%run ./Column_autosize.ipynb
import win32com.client as win32
from jinja2 import Template
import sqlite3
import datetime

## Chargement des données

In [432]:
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str}) # effectif présent
infoc = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\individus_liste_info_complementaires.xlsx", converters={'Code de la ressource':str}) # données extenes
annuaire = pd.read_csv(r"C:\Users\sgasmi\Desktop\Extractions\Export_Annuaire.csv", sep=";", encoding ="ANSI", low_memory=False, error_bad_lines=False) # Extraction de l'annuaire
indentité = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Gestionnaires d'identité.xlsx")
excl_sal = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Salariés à exclure.xlsx")
titre_sej = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Titreséjour.xlsx")
nationality = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Table_Nationalité.xlsx")

In [433]:
## Annuaire, transformation de clé "employeeNumber" en int

def to_int(x):
    try:
        x = int(x)
    except:
        x = 0    
    return x

annuaire["employeeNumber"] = annuaire["employeeNumber"].apply(lambda x: to_int(x))
annuaire = annuaire.loc[annuaire["employeeNumber"]!=0]

In [434]:
excl_sal["Matricule"] = excl_sal["Matricule"].astype("int64")

## Contrôle des données

### Contrôle login SAP

In [435]:
fafr = fafr.merge(infoc[["Code de la ressource", "Login SAP"]], left_on=["Matricule"], right_on=["Code de la ressource"], how='left')

In [436]:
fafr["Login SAP vide"] = ""
fafr.loc[fafr["Login SAP"].isna()==True, "Login SAP vide"] = "Oui"
fafr_sap_ko = fafr.loc[fafr["Login SAP vide"]=="Oui"]
fafr_sap_ko = fafr_sap_ko.merge(indentité, on="Etablissement", how="left")
fafr_sap_ko = fafr_sap_ko.sort_values(by=["Etablissement"])
fafr_sap_ko["Matricule"] =fafr_sap_ko["Matricule"].astype("int64")

In [437]:
fafr_sap_ko = fafr_sap_ko.merge(annuaire[["employeeNumber", "saplogon"]],left_on="Matricule", right_on="employeeNumber", how="left")

In [438]:
fafr_sap_ko = fafr_sap_ko[["Matricule","Nom", "Prénom", "Etablissement","Gestionnaire d'identité", "employeeNumber", "saplogon"]]
fafr_sap_ko = fafr_sap_ko.loc[fafr_sap_ko["Matricule"].isin(excl_sal["Matricule"])==False]
sap_ko_annuaire = fafr_sap_ko.loc[fafr_sap_ko["saplogon"].isna()]

In [439]:
sap_ko_annuaire.loc[sap_ko_annuaire["employeeNumber"].isna()==True, "Causes possibles"] = "Matricule différent de celui du fichier/ Salarié Tiers au lieu d'agent/ Salarié non rattaché à Framatome"
sap_ko_annuaire.loc[sap_ko_annuaire["employeeNumber"].isna()==False, "Causes possibles"] = "Login SAP du compte Agent non renseigné/ Salarié non rattaché à FRAMATOME"

C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

## Contrôle du fichier effectif (FAFR_35)

In [440]:
res_eff = pd.DataFrame(columns=["Matricule", "Nom", "Prénom", "Anomalie", "Matricule Gest Paie", "Nom Gest Paie", "Prénom Gest Paie"])

### Contrôle champs vides obligatoires

In [441]:
fafr = fafr[fafr["Code Contrat"].isin(["STA2", "STA1"])==False].reset_index(drop=True)  # Exclusion des stagiaires
fafr["Position_coef"] = fafr.apply(lambda x: x["Niveau"] if x["Catég conv"]=="C" else x["Niveau"], axis=1) # Ajout du champ position_coef calculé dans l'annuaire

### Vérifications des coefficients vides

In [442]:
res_p = fafr.loc[(fafr["Coeff"].isna()==True) & (fafr["Catég conv"].isin(["D", "CS"])==False)]
res_eff[["Matricule","Nom", "Prénom", "Matricule Gest Paie", "Nom Gest Paie", "Prénom Gest Paie", "Adresse électronique"]] = res_p[["Matricule","Nom", "Prénom", "Matricule Gest Paie", "Nom Gest Paie", "Prénom Gest Paie", "Adresse électronique"]].reset_index(drop=True)
res_eff["Anomalie"] = "Coeff vide"

def to_res(df_to_add, df_to_receive):
    df_to_add = df_to_add[["Matricule","Nom", "Prénom", "Matricule Gest Paie", "Nom Gest Paie", "Prénom Gest Paie", "Adresse électronique","Anomalie"]]
    return df_to_receive.append(df_to_add)


### Vérification des niveaux vides

In [443]:
res_p =  fafr.loc[(fafr["Niveau"].isna()==True) & (fafr["Convention coll."]!="0650") & (fafr["Catég conv"].isin(["D", "CS"])==False)].reset_index(drop=True)
res_p["Anomalie"]= "Niveau vide"

res_eff = to_res(res_p, res_eff)

#print(to_res(res_p, res_eff))

In [444]:
#res_eff.to_excel("C:/Users/sgasmi/Desktop/Export Python/Anomalies/test.xlsx")

### Identification des salariés etrangers

In [445]:
UE = ["DE","BE","BG","HR","DK","ES","FR","GR","NL","IE","IT","LU","MC","NC","PT","CZ","RO","GB","RS","SE","CH","CY","FI","HU","LV","LT","SI"]
nationality_active = nationality.loc[nationality["Date de fin"].isna()==True]
nationality_fr = nationality_active.loc[nationality_active["Pays"]=="FR", "Ressource"]
nationality_fr = nationality_fr.unique()
strangers = nationality_active.loc[(nationality_active["Ressource"].isin(nationality_fr)==False) & (nationality_active["Pays"].isin(UE)==False), "Ressource"].unique()

### Recherche des etrangers sans titres de séjour

In [446]:
salarie_titre = titre_sej["Ressource"].unique()
sans_titre = [sal for sal in strangers if sal not in salarie_titre] # Salariés etrangers n'ayant pas de titre dans PN

In [447]:
salarie_notitre= fafr.loc[fafr["Matricule"].astype("int64").isin(sans_titre)==True]
salarie_notitre["Anomalie"] = "Salarié sans titre de séjour"
res_eff = to_res(salarie_notitre, res_eff)

<ipython-input-447-a7886b3118f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salarie_notitre["Anomalie"] = "Salarié sans titre de séjour"


### Recherche des salariés etrangers ayant un titre expiré

In [448]:
etrangers_titre = [ sal for sal in strangers if sal not in sans_titre] # Exclusion parmis les etrangers des salariés sans titres

In [449]:
etrangers_titre = [sal for sal in etrangers_titre if sal in list(fafr["Matricule"].astype("int64")) ] # On conserve que les salariés présents

In [450]:
titre_etr = titre_sej.loc[titre_sej["Ressource"].isin(etrangers_titre)==True].copy()

In [451]:
titre_etr["today"] = datetime.datetime.today().date()

In [452]:
titre_etr.columns = ['Matricule', 'Date_de_début', 'Date_de_fin', 'Nº_de_carte_étranger',
       'today']

In [453]:
titre_etr.loc[titre_etr["Date_de_fin"].isna()==True, "Date_de_fin"] = datetime.datetime(2050, 1, 1)

In [454]:
# Création d'une base de données in memory
conn = sqlite3.connect(':memory:')

# Ajout de info_comp comme table de la base de données
titre_etr.to_sql('titre_etr', conn, index=False)

req = "Select Matricule, Date_de_début, Date_de_fin, today from titre_etr where today between Date_de_début and Date_de_fin "

In [455]:
# Exécution de la requête et stokage du résultat dans un DataFrame
df = pd.read_sql_query(req, conn)

In [456]:
salarie_off_titre = [sal for sal in titre_etr["Matricule"] if sal not in list(df["Matricule"])]

In [457]:
salarie_exp_titre = fafr.loc[fafr["Matricule"].astype("int64").isin(salarie_off_titre)].copy()
salarie_exp_titre["Anomalie"] = "Titre de séjour expiré"
res_eff = to_res(salarie_exp_titre, res_eff)

### Recherche des salariés sans date de fin

In [458]:
sal_nodate = titre_etr.loc[titre_etr["Date_de_fin"]==datetime.datetime(2050, 1, 1)]
res_nodate = fafr.loc[fafr["Matricule"].astype("int64").isin(list(sal_nodate["Matricule"]))]
res_nodate["Anomalie"] = "Date de fin du titre de séjour absente"
res_eff = to_res(res_nodate, res_eff)

<ipython-input-458-d2efb13af6fb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_nodate["Anomalie"] = "Date de fin du titre de séjour absente"


### Préparation du fichier de synthèse

In [459]:
synth_ano = res_eff.groupby(["Matricule Gest Paie","Nom Gest Paie", "Prénom Gest Paie", "Adresse électronique"])["Matricule"].count().reset_index()
synth_ano = synth_ano.rename(columns={"Matricule": "Nb_anomalies"})

### Découpage des fichiers Excel

In [460]:
rep = r"C:/Users/sgasmi/Desktop/Export Python/Anomalies/"

In [461]:
for gest in synth_ano["Matricule Gest Paie"] :
    exp = res_eff.loc[res_eff["Matricule Gest Paie"]==gest].reset_index(drop=True).copy()
    nom = str(exp.iloc[0,5])
    prenom = str(exp.iloc[0,6])
    nb_ano = exp.iloc[0,4]
    day = str(dt.date.today().day)
    month = str(dt.date.today().month)
    year = str(dt.date.today().year)
    new_rep = rep  + prenom + "_" + nom + "_" + day + month + year + ".xlsx"
    exp.drop(columns=["Matricule Gest Paie","Nom Gest Paie", "Prénom Gest Paie","Adresse électronique"], inplace=True)
    resize_column([exp], ["Anomalie"], new_rep)

### Envoi des mails

In [462]:
with open(r"C:\Users\sgasmi\Desktop\Template\Modèle mail\Mail gestionnaires.html", encoding='utf8') as myfile:
    mail_temp=myfile.read()
t = Template(mail_temp)
html_content=t.render(Nom="Sabri", nb_anomalies=2)
html_content.encode('utf-8')

b'<html>\n  <head> <meta charset="UTF-8">\n  <link rel="stylesheet" type="text/css" href="C:\\Users\\sgasmi\\Desktop\\moncss.css"/>\n\t  <body>\n\t\t  <b1> Bonjour , <br> <br>\n\t\t  \n\t\t  \n\t\t  Dans le cadre de l\'audit de la qualit\xc3\xa9 des donn\xc3\xa9es le , nous avons identifi\xc3\xa9 \n\t\t  \n\t\t  2 anomalies d\xc3\xa9taill\xc3\xa9es\n\t\t  \n\t\t  \n\t\t  dans le fichier en pi\xc3\xa8ce jointe. <br>\n\t\t  Serait-il possible, s\'il te pla\xc3\xaet, d\'effectuer les corrections n\xc3\xa9cessaires pour ces salari\xc3\xa9s? <br>\n\t\t  Dans le cas o\xc3\xb9 ces informations auraient \xc3\xa9t\xc3\xa9 saisies entre temps, merci de ne pas tenir compte de cet email. <br><br>\n\t\t  \n\t\t  Bien cordialement,<br>\n\t\t  \n\t\t  Sabri Gasmi\n\t\t  </b1>\n\t  \n\t  </body>\n  </head>\n</html>'

In [463]:
date_jour = day + '/'+ month +'/'+ year

In [464]:
synth_ano["Prénom Gest Paie"] = synth_ano["Prénom Gest Paie"].apply(lambda x :str(x[0]) + str(x[1:].lower()))

In [465]:
send="ko"
if send =="ok":
    for i, val in enumerate(synth_ano.iterrows()):

        html_content=t.render(Prenom= synth_ano.at[i, "Prénom Gest Paie"], nb_anomalies=synth_ano.at[i, "Nb_anomalies"], Date_jour=date_jour)
        html_content.encode('utf-8')
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.HTMLBody = html_content
        # To attach a file to the email:

        attachment  = "C:/Users/sgasmi/Desktop/Export Python/Anomalies/" + str(synth_ano.at[i, "Prénom Gest Paie"]).upper() + "_"  + str(synth_ano.at[i, "Nom Gest Paie"]) + "_" + day + month + year + ".xlsx"
        mail.Attachments.Add(attachment)

        mail.To = synth_ano.at[i, "Adresse électronique"]
        mail.Subject = 'Anomalies de données People Net'
        mail.Send()

## Export

### Export login SAP KO

In [466]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\SAP_KO.xlsx') as writer:  
    fafr_sap_ko.to_excel(writer, sheet_name='Conso globale', index=False)
    sap_ko_annuaire.to_excel(writer, sheet_name='SAP Manquant dans annuaire', index=False)

### Export Anomalie effectif

In [467]:
resize_column([res_eff],["Anomalies"], r"C:\Users\sgasmi\Desktop\Power BI applications\Sources Anomalies\Anomalie_effectif" + "_" + day + month + year +  ".xlsx")